In [ ]:
import spacy
import collections
from collections import defaultdict
import csv
import glob
import pandas as pd
from os import path

# stripped down spacy pretrained pipe, just tokenizer, lemmatizer, tagger and morphologizer in pipeline
# since it is just used for tokenizing (tokenizer)
# and lemmatizing (tokenizer, lemmatizer, tagger, morphologizer)
nlp = spacy.load(
    "de_core_news_lg", exclude=["tok2vec", "ner", "parser", "attribute_ruler"]
)

In [ ]:
# function to create vocab from csv file, containing tweets
# mode: should lemmas be used instead of token
def create_vocab(file_name, mode=False):
    vocab_count = defaultdict(int)
    # reading file, only using the column containing the documents
    df = pd.read_csv(file_name, sep=",", quoting=csv.QUOTE_NONE, usecols=[2])
    # tokenizing/lemmatizing
    df["text"] = df["text"].apply(lambda x: nlp(str(x)))
    # if token should be used:
    if not mode:
        # output file name
        title = path.basename(file_name)[:-4] + "_vocab_token.csv"
        # count occurences of each unique word
        for line in df["text"]:
            for token in line:
                vocab_count[token.text] += 1

    # if lemma should be used:
    if mode:
        # output file name
        title = path.basename(file_name)[:-4] + "_vocab_lemma.csv"
        # count occurences of each unique lemma
        for line in df["text"]:
            for token in line:
                vocab_count[token.lemma_] += 1

    # sorting by value and filter for min_word_count >= 3
    vocab_count = {
        k: v
        for k, v in sorted(vocab_count.items(), key=lambda item: item[1], reverse=True)
        if v > 2
    }

    # adding rank-column
    df = pd.DataFrame.from_dict(data=vocab_count, orient="index", columns=["count"])
    df["rank"] = range(1, len(df) + 1)

    # writing to file
    df.to_csv(title, header=True, index_label="token")
    return "{title} create with {count} words.".format(
        title=title, count=len(vocab_count)
    )

In [ ]:
for party in glob.glob("../cleaned-data/*.csv"):
    print(create_vocab(party, False))
    print(create_vocab(party, True))

In [ ]:
# merging the vocabs of CDU and CSU to create the Union vocab
# mode: should lemmas be used instead of token (see: create_vocab function)
def create_vocab_union(mode=False):
    vocab_count = defaultdict(int)

    # if token should be used:
    # importing dictionary from create_vocab (above)
    if not mode:
        title = "union_vocab_token.csv"
        with open("CDU_vocab_token.csv", mode="r") as infile:
            reader = csv.reader(infile)
            # skip first line
            next(reader)
            cdu = {rows[0]: rows[1] for rows in reader}
        with open("CSU_vocab_token.csv", mode="r") as infile:
            reader = csv.reader(infile)
            # skip first line
            next(reader)
            csu = {rows[0]: rows[1] for rows in reader}

    # if lemma should be used:
    # importing dictionary from create_vocab (above)
    if mode:
        title = "union_vocab_lemma.csv"
        with open("CDU_vocab_lemma.csv", mode="r") as infile:
            reader = csv.reader(infile)
            # skip first line
            next(reader)
            cdu = {rows[0]: rows[1] for rows in reader}
        with open("CSU_vocab_lemma.csv", mode="r") as infile:
            reader = csv.reader(infile)
            # skip first line
            next(reader)
            csu = {rows[0]: rows[1] for rows in reader}

    # merging
    for party in (cdu, csu):
        for item in party:
            vocab_count[item] += int(party[item])
    # sorting by value and filter for min_word_count >= 3
    vocab_count = {
        k: v
        for k, v in sorted(vocab_count.items(), key=lambda item: item[1], reverse=True)
        if v > 2
    }

    # adding rank-column
    df = pd.DataFrame.from_dict(data=vocab_count, orient="index", columns=["count"])
    df["rank"] = range(1, len(df) + 1)

    # writing to file
    df.to_csv(title, header=True, index_label="token")
    return "{title} create with {count} words.".format(
        title=title, count=len(vocab_count)
    )

In [ ]:
# merging cdu and csu
print(create_vocab_union(False))
print(create_vocab_union(True))